In [29]:
import math
import pandas as pd
import numpy as np
from numpy.random import randn
import random

train_data = {
  'good': True,
  'bad': False,
  'happy': True,
  'sad': False,
  'not good': False,
  'not bad': True,
  'not happy': False,
  'not sad': True,
  'very good': True,
  'very bad': False,
  'very happy': True,
  'very sad': False,
  'i am happy': True,
  'this is good': True,
  'i am bad': False,
  'this is bad': False,
  'i am sad': False,
  'this is sad': False,
  'i am not happy': False,
  'this is not good': False,
  'i am not bad': True,
  'this is not sad': True,
  'i am very happy': True,
  'this is very good': True,
  'i am very bad': False,
  'this is very sad': False,
  'this is very happy': True,
  'i am good not bad': True,
  'this is good not bad': True,
  'i am bad not good': False,
  'i am good and happy': True,
  'this is not good and not happy': False,
  'i am not at all good': False,
  'i am not at all bad': True,
  'i am not at all happy': False,
  'this is not at all sad': True,
  'this is not at all happy': False,
  'i am good right now': True,
  'i am bad right now': False,
  'this is bad right now': False,
  'i am sad right now': False,
  'i was good earlier': True,
  'i was happy earlier': True,
  'i was bad earlier': False,
  'i was sad earlier': False,
  'i am very bad right now': False,
  'this is very good right now': True,
  'this is very sad right now': False,
  'this was bad earlier': False,
  'this was very good earlier': True,
  'this was very bad earlier': False,
  'this was very happy earlier': True,
  'this was very sad earlier': False,
  'i was good and not bad earlier': True,
  'i was not good and not happy earlier': False,
  'i am not at all bad or sad right now': True,
  'i am not at all good or happy right now': False,
  'this was not happy and not good earlier': False,
}

test_data = {
  'this is happy': True,
  'i am good': True,
  'this is not happy': False,
  'i am not good': False,
  'this is not bad': True,
  'i am not sad': True,
  'i am very good': True,
  'this is very bad': False,
  'i am very sad': False,
  'this is bad not good': False,
  'this is good and happy': True,
  'i am not good and not happy': False,
  'i am not at all sad': True,
  'this is not at all good': False,
  'this is not at all bad': True,
  'this is good right now': True,
  'this is sad right now': False,
  'this is very bad right now': False,
  'this was good earlier': True,
  'i was not happy and not good earlier': False,
}

In [30]:
class RNN:
  # A many-to-one Vanilla Recurrent Neural Network.

  def __init__(self, input_size, output_size, hidden_size=5):
    # Weights
    self.Whh = randn(hidden_size, hidden_size) / 1000
    self.Wxh = randn(hidden_size, input_size) / 1000
    self.Why = randn(output_size, hidden_size) / 1000

    # Biases
    self.bh = np.zeros((hidden_size, 1))
    self.by = np.zeros((output_size, 1))

    self.h = np.zeros((self.Whh.shape[0], 1))

  def forward(self, inputs):
    '''
    Perform a forward pass of the RNN using the given inputs.
    Returns the final output and hidden state.
    - inputs is an array of one hot vectors with shape (input_size, 1).
    '''
    h = np.zeros((self.Whh.shape[0], 1))

    self.last_inputs = inputs
    self.last_hs = { 0: h }
    
    # Perform each step of the RNN
    for i, x in enumerate(inputs):
      h = np.tanh(self.Wxh @ x + self.Whh @ h + self.bh)
      self.last_hs[i + 1] = h

    # Compute the output
    y = self.Why @ h + self.by
    #print((self.last_hs))
    return y, h

  def backprop(self, d_y, learn_rate=2e-2):
    '''
    Perform a backward pass of the RNN.
    - d_y (dL/dy) has shape (output_size, 1).
    - learn_rate is a float.
    '''
    n = len(self.last_inputs)

    # Calculate dL/dWhy and dL/dby.
    d_Why = d_y @ self.last_hs[n].T
    d_by = d_y

    # Initialize dL/dWhh, dL/dWxh, and dL/dbh to zero.
    d_Whh = np.zeros(self.Whh.shape)
    d_Wxh = np.zeros(self.Wxh.shape)
    d_bh = np.zeros(self.bh.shape)

    # Calculate dL/dh for the last h.
    # dL/dh = dL/dy * dy/dh
    d_h = self.Why.T @ d_y

    # Backpropagate through time.
    for t in reversed(range(n)):
      # An intermediate value: dL/dh * (1 - h^2)
      temp = ((1 - self.last_hs[t + 1] ** 2) * d_h)

      # dL/db = dL/dh * (1 - h^2)
      d_bh += temp

      # dL/dWhh = dL/dh * (1 - h^2) * h_{t-1}
      d_Whh += temp @ self.last_hs[t].T

      # dL/dWxh = dL/dh * (1 - h^2) * x
      d_Wxh += temp @ self.last_inputs[t].T

      # Next dL/dh = dL/dh * (1 - h^2) * Whh
      d_h = self.Whh @ temp

    # Clip to prevent exploding gradients.
    for d in [d_Wxh, d_Whh, d_Why, d_bh, d_by]:
      np.clip(d, -1, 1, out=d)

    # Update weights and biases using gradient descent.
    self.Whh -= learn_rate * d_Whh
    self.Wxh -= learn_rate * d_Wxh
    self.Why -= learn_rate * d_Why
    self.bh -= learn_rate * d_bh
    self.by -= learn_rate * d_by

  
  def adapt(self,x):
    h = np.tanh(self.Wxh @ x + self.Whh @ self.h + self.bh)

In [ ]:
# Create the vocabulary.
vocab = list(set([w for text in train_data.keys() for w in text.split(' ')]))
vocab_size = len(vocab)
print('%d unique words found' % vocab_size)

# Assign indices to each word.
word_to_idx = { w: i for i, w in enumerate(vocab) }
idx_to_word = { i: w for i, w in enumerate(vocab) }
# print(word_to_idx['good'])
# print(idx_to_word[0])

def createInputs(text):
  '''
  Returns an array of one-hot vectors representing the words in the input text string.
  - text is a string
  - Each one-hot vector has shape (vocab_size, 1)
  '''
  inputs = []
  for w in text.split(' '):
    v = np.zeros((vocab_size, 1))
    v[word_to_idx[w]] = 1
    inputs.append(v)
  return inputs

def softmax(xs):
  # Applies the Softmax Function to the input array.
  return np.exp(xs) / sum(np.exp(xs))

# Initialize our RNN!
rnn = RNN(vocab_size, 2)

def processData(data, backprop=True):
  '''
  Returns the RNN's loss and accuracy for the given data.
  - data is a dictionary mapping text to True or False.
  - backprop determines if the backward phase should be run.
  '''
  items = list(data.items())
  random.shuffle(items)

  loss = 0
  num_correct = 0

  for x, y in items:
    inputs = createInputs(x)
    target = int(y)
    # Forward
    #print(len(inputs))
    
    out, _ = rnn.forward(inputs)
    probs = softmax(out)

    # Calculate loss / accuracy
    loss -= np.log(probs[target])
    num_correct += int(np.argmax(probs) == target)

    if backprop:
      # Build dL/dy
      d_L_d_y = probs
      d_L_d_y[target] -= 1

      # Backward
      rnn.backprop(d_L_d_y)

  return loss / len(data), num_correct / len(data)

# Training loop
for epoch in range(1):
  train_loss, train_acc = processData(train_data)

  if epoch % 100 == 99:
    print('--- Epoch %d' % (epoch + 1))
    print('Train:\tLoss %.3f | Accuracy: %.3f' % (train_loss, train_acc))

    test_loss, test_acc = processData(test_data, backprop=False)
    print('Test:\tLoss %.3f | Accuracy: %.3f' % (test_loss, test_acc))

In [74]:
import numpy as np

class RTLO:
    def __init__(self, n_in, n_rec, n_out,N1,N2,N3, tau_m=1,decay=1e-5):
        np.random.seed(42)
        self.n_in = n_in
        self.n_rec = n_rec
        self.n_out = n_out
        self.xI = np.zeros(n_in)
        self.tau_m = tau_m
        self.n_spl = 1
        self.ni = np.array([N1,N2,N3])
        self.ni0 = np.array([N1,N2,N3])
        self.T = 1
        self.mErr = 0
        self.dw_in = 0
        self.dw_rec = 0
        self.dw_out = 0
        self.k = 0
        self.decay = decay
        self.ht = 0
        self.wit = 0
        self.wrt = 0
        self.wot = 0
    
        #self.w_in = np.zeros([n_rec, n_in])+1e-6
        #self.w_rec = np.zeros([n_rec, n_rec])+1e-6
        #self.w_out = np.zeros([n_out, n_rec])+1e-6
        #self.b = np.zeros([n_rec, n_out])+1e-6

        self.w_in = self.xavier_uniform([n_rec, n_in])
        self.w_rec = self.xavier_uniform([n_rec, n_rec])
        self.w_out = self.xavier_uniform([n_out, n_rec])
        self.b = np.random.randn(n_rec, n_out)/n_out**0.5

        self.u = 0
        self.uI = 0
        self.hI = np.zeros(self.n_rec)
        self.hF = np.zeros(self.n_rec)

        self.p = np.zeros((self.n_rec, self.n_rec))
        self.q = np.zeros((self.n_rec, self.n_in))

        self.j = 1
        #self.b = np.random.randn(n_rec, n_out)/n_out**0.5


    def adapt(self, x, yR,tip=1,parameters=False):
        self.ni = self.ni0/(1 + self.decay*self.k)
        N1, N2, N3 = self.ni[0], self.ni[1], self.ni[2]

        self.u= np.dot(self.w_rec, self.hI) + np.dot(self.w_in, x)
        self.hF = self.hI + (-self.hI + self.phi(self.u))/self.tau_m
        yP = np.dot(self.w_out, self.hF)
        err = yR - yP  # readout error
        self.p = np.outer(self.dphi(self.uI),self.hI)/self.tau_m + (1-1/self.tau_m)*self.p

        self.q = np.outer(self.dphi(self.uI),self.xI)/self.tau_m + (1-1/self.tau_m)*self.q

        #self.dw_out = (eta1*np.outer(err, self.hF))
        #self.dw_rec = eta2*np.outer(np.dot(self.b, err),np.ones(self.n_rec))*self.p
        #self.dw_in = eta3*np.outer(np.dot(self.b, err),np.ones(self.n_in))*self.q
        
        self.dw_out = (self.dw_out*(self.T-1)/self.T)+((N1*np.outer(err, self.hF))/self.T)*tip
        self.dw_rec = (self.dw_rec*(self.T-1)/self.T)+(N2*np.outer(np.dot(self.b, err),np.ones(self.n_rec))*self.p/self.T)*tip
        self.dw_in = (self.dw_in*(self.T-1)/self.T)+(N3*np.outer(np.dot(self.b, err),np.ones(self.n_in))*self.q/self.T)*tip

        self.w_out = self.w_out + self.dw_out
        self.w_rec = self.w_rec + self.dw_rec
        self.w_in = self.w_in + self.dw_in

        if parameters:
            #print('x:',x,'y:',yR)
            #print('u:',self.u)
            print('hI:',self.hI)
            '''print('hF:',self.hF)
            print('yP:',yP)
            print('err:',err)
            print('p:', self.p)
            print('q:', self.q)
            print("self.dw_out:", self.dw_out)
            print("self.dw_rec:", self.dw_rec)
            print("self.dw_in:", self.dw_in)
            print('w_out:',self.w_out)
            print('w_rec:',self.w_rec)
            print('w_in:',self.w_in)'''

        self.hI[]

        self.ht = self.hF
        self.ht2 = self.hF
        self.wit = self.w_in
        self.wrt = self.w_rec
        self.wot = self.w_out
        #self.hI = self.hF
        self.xI = x
        self.uI = self.u
        self.T = self.T+1  
        self.k = self.k+1 
        return
    
    def predict(self,x):
        #print(self.ht)
        #print('.......')
        u = np.dot(self.wrt, self.ht) + np.dot(self.wit, x)
        h = self.ht + (-self.ht + self.phi(u))/self.tau_m
        y = np.dot(self.wot, h)
        self.ht = h
        return y
    
    def predict2(self,x):
        #print(self.ht)
        #print('.......')
        u = np.dot(self.wrt, self.ht2) + np.dot(self.wit, x)
        h = self.ht2 + (-self.ht2 + self.phi(u))/self.tau_m
        y = np.dot(self.wot, h)
        self.ht2 = h
        return y
    
    def restore(self):
        self.ht = self.hF
        self.ht2 = self.hF

    def restore_ni(self):
        self.ni = self.ni0
        self.k = 0
        self.T = 1
    
    def restore_ni2(self):
        self.ni = self.ni0
        self.k = 0
        #self.T = 1

    def phi(self,x):
        return np.tanh(x)
    
    def dphi(self,x):
        return 1/np.cosh(10*np.tanh(x/10))**2  # the tanh prevents oveflow
    
    def xavier_uniform(self,shape):
        np.random.seed(42)
        n_in, n_out = shape
        limit = np.sqrt(6 / (n_in + n_out))
        return np.random.uniform(-limit, limit, size=shape)



SyntaxError: invalid syntax (322018032.py, line 88)

In [72]:
path = r"C:\Artigo_J3C_V2\VMD10\HI\Bearing1_1.csv"
df = pd.read_csv(path)
sig = df['HIX'].values

n=5
nIN = 4
nOUT = 3
data = np.array([sig[i:i+n] for i in range(len(sig)-n)])
X = (data.T[:nIN]).T
Y = (data.T[-nOUT:]).T

In [78]:
rnn = RTLO(n_in=nIN,n_rec=5,n_out=nOUT,N1=1,N2=1,N3=1,tau_m=1)
cont = 0
for x,y in zip(X,Y):
    rnn.adapt(x,y,parameters=True)
    cont = cont +1
    if cont>3: break

hI: [0. 0. 0. 0. 0.]
hI: [0. 0. 0. 0. 0.]
hI: [0. 0. 0. 0. 0.]
hI: [0. 0. 0. 0. 0.]
hI: [0. 0. 0. 0. 0.]
hI: [ 0.71745211 -0.78209751  0.39040272 -0.67050656 -0.54840612]
hI: [ 0.71745211 -0.78209751  0.39040272 -0.67050656 -0.54840612]
hI: [ 0.71745211 -0.78209751  0.39040272 -0.67050656 -0.54840612]
hI: [ 0.71745211 -0.78209751  0.39040272 -0.67050656 -0.54840612]
hI: [ 0.71745211 -0.78209751  0.39040272 -0.67050656 -0.54840612]
hI: [ 0.99999998 -0.88822594  0.99999825  0.99994069 -1.        ]
hI: [ 0.99999998 -0.88822594  0.99999825  0.99994069 -1.        ]
hI: [ 0.99999998 -0.88822594  0.99999825  0.99994069 -1.        ]
hI: [ 0.99999998 -0.88822594  0.99999825  0.99994069 -1.        ]
hI: [ 0.99999998 -0.88822594  0.99999825  0.99994069 -1.        ]
hI: [ 1.         -0.85481724  0.99999997  0.99999991 -1.        ]
hI: [ 1.         -0.85481724  0.99999997  0.99999991 -1.        ]
hI: [ 1.         -0.85481724  0.99999997  0.99999991 -1.        ]
hI: [ 1.         -0.85481724  0.99999